# Data discovery with NASA's CMR

## Summary

In this notebook, we will walk through how to search for Earthdata data collections and granules. Along the way we will explore the available search parameters, information return, and specific contrains when using the CMR API. Our object is to identify assets to access that we would downloaded, or perform S3 direct access, within an analysis workflow 

We will be querying CMR for [ECOSTRESS]() collections/granules to identify assets we would downloaded, or perform S3 direct access, within an analysis workflow

## Requirements

### 1. Earthdata Login

An Earthdata Login account is required to access data, as well as discover restricted data, from the NASA Earthdata system. Thus, to access NASA data, you need Earthdata Login. Please visit https://urs.earthdata.nasa.gov to register and manage your Earthdata Login account. This account is free to create and only takes a moment to set up.

### 2. ECOSTRESS Early Adopter

ECOSTRESS build 7 is only open to individuals identified as early adopters. As such ECOSTRESS discovery and access is managed by an access control list. If you are not on the access control list, you will not be able to complete the exercise as written below. 

## Learning Objectives

- understand what CMR/CMR API is and what CMR/CMR API can be used for 
- how to use the `requests` package to search data collections and granules
- how to use an Earthdata Login token to search for data with access control lists
- how to parse the results of these searches.

## What is CMR
CMR is the Common Metadata Repository.  It catalogs all data for NASA's Earth Observing System Data and Information System (EOSDIS).  It is the backend of [Earthdata Search](https://search.earthdata.nasa.gov/search), the GUI search interface you are probably familiar with.  More information about CMR can be found [here](https://earthdata.nasa.gov/eosdis/science-system-description/eosdis-components/cmr).

Unfortunately, the GUI for Earthdata Search is not accessible from a cloud instance - at least not without some work.  Earthdata Search is also not immediately reproducible.  What I mean by that is if you create a search using the GUI you would have to note the search criteria (date range, search area, collection name, etc), take a screenshot, copy the search url, or save the list of data granules returned by the search, in order to recreate the search.  This information would have to be re-entered each time you or someone else wanted to do the search.  You could make typos or other mistakes.  A cleaner, reproducible solution is to search CMR programmatically using the CMR API.

## What is the CMR API
API stands for Application Programming Interface.  It allows applications (software, services, etc) to send information to each other.  A helpful analogy is a waiter in a restaurant.  The waiter takes your drink or food order that you select from the menu, often translated into short-hand, to the bar or kitchen, and then returns (hopefully) with what you ordered when it is ready.

The CMR API accepts search terms such as collection name, keywords, datetime range, and location, queries the CMR database and returns the results.

---

## Getting Started: How to search CMR from Python
The first step is to import python packages.  We will use:  
- `requests` This package does most of the work for us accessing the CMR API using HTTP methods. 
- `pprint` to _pretty print_ the results of the search.  

A more in-depth tutorial on `requests` is [here](https://realpython.com/python-requests/)

In [1]:
import requests
import json
from pprint import pprint

To conduct a search using the CMR API, `requests` needs the url for the root CMR search endpoint. We'll assign this url to a python variable as a _string_.

In [2]:
CMR_OPS = 'https://cmr.earthdata.nasa.gov/search'

CMR allows search by __collections__, which are datasets, and __granules__, which are files that contain data. Many of the same search parameters can be used for collections and granules but the type of results returned differ. Search parameters can be found in the [API Documentation](https://cmr.earthdata.nasa.gov/search/site/docs/search/api.html).  

Whether we search __collections__ or __granules__ is distinguished by adding `"collections"` or `"granules"` to the end of the CMR endpoint URL.  

We are going to search collections first, so we add `"collections"` to the URL. We are using a `python` format string in the examples below.

In [3]:
url = f'{CMR_OPS}/{"collections"}'

In this first example, I want to retrieve a list of __ECOSTRESS__ collections in the Earthdata Cloud. This includes ECOSTRESS collections from build 7 which at the time of this tutorial, is hidden to all except early adopters. Because of this, an extra parameter needs to be passed in each CMR request that indicates you are part of the access list. An Earthdata Login token will be passed to the `token` parameter, which is generated using your Earthdata Login credentials.  

Two options are available to generate an Earthdata Login token. 
1. Generate a token from the Earthdata Login interface by logging into Earthdata Login and Click Generate Token.
2. Programatically generate an Earthdata Login token. Use the [NASA_Earthdata_Login_Token](../how-tos/authentication/NASA_Earthdata_Login_Token.ipynb) notebook to generate and save a token for use in this notebook.  

We can read in our token after it has been generated and saved using the [NASA_Earthdata_Login_Token](../how-tos/authentication/NASA_Earthdata_Login_Token.ipynb) notebook. The json file produce can be found here: `/home/jovyan/.hidden_dir/edl_token.json`. We'll read to token into a variable named `token`.

In [4]:
with open('../../.hidden_dir/edl_token.json') as js:
    token = json.load(js)['access_token']

We'll want to get the content in `json` (pronounced "jason") format, so I pass a dictionary to the header keyword argument to say that I want results returned as `json`.

The `.get()` method is used to send this information to the CMR API. `get()` calls the HTTP method __GET__. 

In [5]:
response = requests.get(url,
                        params={
                            'cloud_hosted': 'True',
                            'has_granules': 'True',
                        },
                        headers={
                            'Accept': 'application/json',
                        }
                       )

The request returns a `Response` object.    

To check that our request was successful we can print the `response` variable we saved the request to.

In [6]:
response

<Response [200]>

A __200__ response is what we want. This means that the requests was successful. For more information on HTTP status codes see <https://en.wikipedia.org/wiki/List_of_HTTP_status_codes>

A more explict way to check the status code is to use the `status_code` attribute. Both methods return a HTTP status code.

In [7]:
response.status_code

200

The response from `requests.get` returns the results of the search and metadata about those results in the `headers`.  

More information about the `response` object can be found by typing `help(response)`.

`headers` contains useful information in a case-insensitive dictionary. We requested (above) that the information be return in json which means the object return is a dictionary in our Python environment. We'll iterate through the returned dictionary, looping throught each field (`k`) and its associated value (`v`). For more on interating through dictionary object click [here](https://realpython.com/iterate-through-dictionary-python/).

In [8]:
for k, v in response.headers.items():
    print(f'{k}: {v}')

Content-Type: application/json;charset=utf-8
Content-Length: 4518
Connection: keep-alive
Date: Fri, 08 Apr 2022 17:26:19 GMT
X-Frame-Options: SAMEORIGIN
Access-Control-Allow-Origin: *
X-XSS-Protection: 1; mode=block
CMR-Request-Id: 4af129b5-7119-4c6c-8772-3b5dab7ac964
Strict-Transport-Security: max-age=31536000
CMR-Search-After: [0.0,18400.0,"SENTINEL-1A_OCN","1",1214472977,1250]
CMR-Hits: 1172
Access-Control-Expose-Headers: CMR-Hits, CMR-Request-Id, X-Request-Id, CMR-Scroll-Id, CMR-Search-After, CMR-Timed-Out, CMR-Shapefile-Original-Point-Count, CMR-Shapefile-Simplified-Point-Count
X-Content-Type-Options: nosniff
CMR-Took: 194
X-Request-Id: pW-PuXNWhMvuUsomy4h5mpMBxPIZQ1A4fuSGh_DZLtQL_y6fSrAZkQ==
Vary: Accept-Encoding, User-Agent
Content-Encoding: gzip
Server: ServerTokens ProductOnly
X-Cache: Miss from cloudfront
Via: 1.1 e9c8cd6cad69627cb7c9d88123e6e2cc.cloudfront.net (CloudFront)
X-Amz-Cf-Pop: HIO50-C2
X-Amz-Cf-Id: pW-PuXNWhMvuUsomy4h5mpMBxPIZQ1A4fuSGh_DZLtQL_y6fSrAZkQ==


Each item in the dictionary can be accessed in the normal way you access a `python` dictionary but the keys uniquely case-insensitive. Let's take a look at the commonly used `CMR-Hits` key.



In [9]:
response.headers['CMR-Hits']

'1172'

Note that "cmr-hits" works as well!

In [10]:
response.headers['cmr-hits']

'1172'

In some situations the response to your query can return a very large number of result, some of which may not be relevant. We can add additional [query parameters](https://cmr.earthdata.nasa.gov/search/site/docs/search/api.html) to restrict the information returned. We're going to restrict the search by the `provider` parameter.

You can modify the code below to explore all Earthdata data products hosted by the various providers. When searching by provider, use _Cloud Provider_ to search for cloud-hosted datasets and _On-Premises Provider_ to search for datasets archived at the DAACs. A partial list of providers is given below.

DAAC      | Short Name                              | Cloud Provider | On-Premises Provider  
----------|-----------------------------------------|----------------|----------------------  
NSIDC     | National Snow and Ice Data Center       | NSIDC_CPRD     | NSIDC_ECS  
GHRC DAAC | Global Hydrometeorology Resource Center | GHRC_DAAC      | GHRC_DAAC  
PO DAAC   | Physical Oceanography Distributed Active Archive Center | POCLOUD | PODAAC  
ASF       | Alaska Satellite Facility | ASF | ASF  
ORNL DAAC | Oak Ridge National Laboratory | ORNL_CLOUD | ORNL_DAAC  
LP DAAC   | Land Processes Distributed Active Archive Center | LPCLOUD | LPDAAC_ECS
GES DISC  | NASA Goddard Earth Sciences (GES) Data and Information Services Center (DISC) | GES_DISC | GES_DISC
OB DAAC   | NASA's Ocean Biology Distributed Active Archive Center |   | OB_DAAC
SEDAC     | NASA's Socioeconomic Data and Applications Center |   | SEDAC

We'll assign the provider to a variable as a _string_ and insert the variable into the parameter argument in the request. We'll also assign the term 'ECOSTRESS' to a varible so we don't need to repeatedly add it to the requests parameters. 

In [11]:
provider = 'LPCLOUD'
project = 'ECOSTRESS'

In [12]:
headers = {
    'Authorization': f'Bearer {token}',
    'Accept': 'application/json',
}

In [13]:
response = requests.get(url,
                        params={
                            'cloud_hosted': 'True',
                            'has_granules': 'True',
                            'provider': provider,
                            'project': project,
                        },
                        headers=headers
                       )
response

<Response [200]>

In [14]:
response.headers['cmr-hits']

'13'

Search results are contained in the __content__ part of the Response object. However, `response.content` returns information in bytes.

In [15]:
response.content

b'{"feed":{"updated":"2022-04-08T17:26:20.222Z","id":"https://cmr.earthdata.nasa.gov:443/search/collections.json?cloud_hosted=True&has_granules=True&provider=LPCLOUD&project=ECOSTRESS","title":"ECHO dataset metadata","entry":[{"processing_level_id":"1A","boxes":["-90 -180 90 180"],"time_start":"2014-01-01T00:00:00.000Z","version_id":"002","updated":"2021-06-23T16:51:15.164Z","dataset_id":"ECOSTRESS Swath Black Body Instantaneous L1A Global 38 x 69 m V002","has_spatial_subsetting":false,"has_transforms":false,"has_variables":false,"data_center":"LPCLOUD","short_name":"ECO_L1A_BB","organizations":["LP DAAC","NASA/JPL/ECOSTRESS"],"title":"ECOSTRESS Swath Black Body Instantaneous L1A Global 38 x 69 m V002","coordinate_system":"CARTESIAN","summary":"The ECOsystem Spaceborne Thermal Radiometer Experiment on Space Station (ECOSTRESS) will measure the temperature of plants and use that information to better understand how much water plants need and how they respond to stress.","service_feature

A more convenient way to work with this information is to use `json` formatted data. I'm using pretty print `pprint` to print the data in an easy to read way.    

**Note**
- `response.json()` will format our response in `json` 
- `['feed']['entry']` returns all entries that CMR returned in the request (not the same as __CMR-Hits__)
- `[0]` returns the first entry. Reminder that python starts indexing at 0, not 1!

In [16]:
pprint(response.json()['feed']['entry'][0])

{'archive_center': 'LP DAAC',
 'boxes': ['-90 -180 90 180'],
 'browse_flag': False,
 'collection_data_type': 'SCIENCE_QUALITY',
 'consortiums': ['GEOSS', 'EOSDIS'],
 'coordinate_system': 'CARTESIAN',
 'data_center': 'LPCLOUD',
 'dataset_id': 'ECOSTRESS Swath Black Body Instantaneous L1A Global 38 x 69 m '
               'V002',
 'has_formats': False,
 'has_spatial_subsetting': False,
 'has_temporal_subsetting': False,
 'has_transforms': False,
 'has_variables': False,
 'id': 'C2076119270-LPCLOUD',
 'links': [{'href': 'https://search.earthdata.nasa.gov/search?q=ECO1ABB+V001',
            'hreflang': 'en-US',
            'rel': 'http://esipfed.org/ns/fedsearch/1.1/data#'},
           {'href': 'https://e4ftl01.cr.usgs.gov/ECOSTRESS/ECO1ABB.001/',
            'hreflang': 'en-US',
            'rel': 'http://esipfed.org/ns/fedsearch/1.1/data#'},
           {'href': 'https://lpdaac.usgs.gov/',
            'hreflang': 'en-US',
            'rel': 'http://esipfed.org/ns/fedsearch/1.1/metadata#'}

The first response contains a lot more information than we need. We'll narrow in on a few fields to get a feel for what we have. We'll print the name of the dataset (`dataset_id`) and the concept id (`id`). We can build this variable and print statement like we did above with the `url` variable. 

In [17]:
collections = response.json()['feed']['entry']

In [18]:
for collection in collections:
    print(f'{collection["archive_center"]} | {collection["dataset_id"]} | {collection["id"]}')

LP DAAC | ECOSTRESS Swath Black Body Instantaneous L1A Global 38 x 69 m V002 | C2076119270-LPCLOUD
LP DAAC | ECOSTRESS Swath Engineering Data Instantaneous L1A Global V002 | C2076086331-LPCLOUD
LP DAAC | ECOSTRESS Swath Pixel Data Instantaneous L1A Global 38 x 69 m V002 | C2076118669-LPCLOUD
LP DAAC | ECOSTRESS Swath Attitude and Ephemeris Instantaneous L1B Global V002 | C2076117996-LPCLOUD
LP DAAC | ECOSTRESS Swath Geolocation Instantaneous L1B Global 70 m V002 | C2076087338-LPCLOUD
LP DAAC | ECOSTRESS Swath Top of Atmosphere Calibrated Radiance Instantaneous L1B Global 70 m V002 | C2076116385-LPCLOUD
LP DAAC | ECOSTRESS Gridded Cloud Mask Instantaneous L2 Global 70 m V002 | C2076113561-LPCLOUD
LP DAAC | ECOSTRESS Gridded Land Surface Temperature and Emissivity Instantaneous L2 Global 70 m V002 | C2076113037-LPCLOUD
LP DAAC | ECOSTRESS Gridded Top of Atmosphere Calibrated Radiance Instantaneous L2 Global 70 m V002 | C2074824682-LPCLOUD
LP DAAC | ECOSTRESS Tiled Land Surface Temperatur

In some situations we may be expecting a certain number of results. Note here that we only have 10 datasets are printed. We know from `CMR-Hits` that there are more than 10 datasets. This is because CMR restricts the number of results returned by each query. The default is 10 but it can be set to a maximum of 2000. We'll set the `page_size` parameter to 25 so we return all results in a single query.

In [19]:
response = requests.get(url,
                        params={
                            'cloud_hosted': 'True',
                            'has_granules': 'True',
                            'provider': provider,
                            'project': project,
                            'page_size': 25
                        },
                        headers=headers
                       )
response

<Response [200]>

In [20]:
response.headers['cmr-hits']

'13'

Now, when we can re-run our for loop for the collections we now have all of the available collections listed.

In [21]:
collections = response.json()['feed']['entry']
for collection in collections:
    print(f'{collection["archive_center"]} | {collection["dataset_id"]} | {collection["id"]}')

LP DAAC | ECOSTRESS Swath Black Body Instantaneous L1A Global 38 x 69 m V002 | C2076119270-LPCLOUD
LP DAAC | ECOSTRESS Swath Engineering Data Instantaneous L1A Global V002 | C2076086331-LPCLOUD
LP DAAC | ECOSTRESS Swath Pixel Data Instantaneous L1A Global 38 x 69 m V002 | C2076118669-LPCLOUD
LP DAAC | ECOSTRESS Swath Attitude and Ephemeris Instantaneous L1B Global V002 | C2076117996-LPCLOUD
LP DAAC | ECOSTRESS Swath Geolocation Instantaneous L1B Global 70 m V002 | C2076087338-LPCLOUD
LP DAAC | ECOSTRESS Swath Top of Atmosphere Calibrated Radiance Instantaneous L1B Global 70 m V002 | C2076116385-LPCLOUD
LP DAAC | ECOSTRESS Gridded Cloud Mask Instantaneous L2 Global 70 m V002 | C2076113561-LPCLOUD
LP DAAC | ECOSTRESS Gridded Land Surface Temperature and Emissivity Instantaneous L2 Global 70 m V002 | C2076113037-LPCLOUD
LP DAAC | ECOSTRESS Gridded Top of Atmosphere Calibrated Radiance Instantaneous L2 Global 70 m V002 | C2074824682-LPCLOUD
LP DAAC | ECOSTRESS Tiled Land Surface Temperatur

## Searching for Granules
In NASA speak, Granules are files or groups of files. In this example, we will search for ECO2LSTE version 1 for a specified region of interest and datetime range.  

We need to change the resource url to look for __granules__ instead of collections

In [22]:
url = f'{CMR_OPS}/{"granules"}'

We will search by `concept_id`, `temporal`, and `bounding_box`.  Details about these search parameters can be found in the CMR API Documentation.

The formatting of the values for each parameter is quite specific.  
__Temporal parameters__ are in ISO 8061 format `yyyy-MM-ddTHH:mm:ssZ`.  
__Bounding box coordinates__ are lower left longitude, lower left latitude, upper right longitude, upper right latitude. 

In [23]:
collection_id = 'C2076090826-LPCLOUD'
date_range = '2022-04-01T00:00:00Z,2022-04-30T23:59:59Z'
bbox = '-120.45264628,34.51050622,-120.40432448,34.53239876'

In [24]:
response = requests.get(url, 
                        params={
                            'concept_id': collection_id,
                            'temporal': date_range,
                            'bounding_box': bbox,
                            'token': token,
                            'page_size': 200
                            },
                        headers=headers
                       )
print(response.status_code)

200


In [25]:
print(response.headers['CMR-Hits'])

1


In [26]:
granules = response.json()['feed']['entry']
for granule in granules:
    print(f'{granule["data_center"]} | {granule["dataset_id"]} | {granule["id"]}')

LPCLOUD | ECOSTRESS Tiled Land Surface Temperature and Emissivity Instantaneous L2 Global 70 m V002 | G2248838693-LPCLOUD


In [27]:
pprint(granules[0])

{'boxes': ['34.207188 -120.828926 35.223133 -119.598442'],
 'browse_flag': True,
 'collection_concept_id': 'C2076090826-LPCLOUD',
 'coordinate_system': 'GEODETIC',
 'data_center': 'LPCLOUD',
 'dataset_id': 'ECOSTRESS Tiled Land Surface Temperature and Emissivity '
               'Instantaneous L2 Global 70 m V002',
 'day_night_flag': 'NIGHT',
 'granule_size': '13.9612',
 'id': 'G2248838693-LPCLOUD',
 'links': [{'href': 'https://data.lpdaac.earthdatacloud.nasa.gov/lp-prod-protected/ECO_L2T_LSTE.002/ECOv002_L2T_LSTE_21241_015_10SGD_20220405T034009_0700_01/ECOv002_L2T_LSTE_21241_015_10SGD_20220405T034009_0700_01_water.tif',
            'hreflang': 'en-US',
            'rel': 'http://esipfed.org/ns/fedsearch/1.1/data#',
            'title': 'Download '
                     'ECOv002_L2T_LSTE_21241_015_10SGD_20220405T034009_0700_01_water.tif'},
           {'href': 's3://lp-prod-protected/ECO_L2T_LSTE.002/ECOv002_L2T_LSTE_21241_015_10SGD_20220405T034009_0700_01/ECOv002_L2T_LSTE_21241_015_10SG

## Get URLs to cloud data assets

In [28]:
https_urls = [l['href'] for l in granules[0]['links'] if 'https' in l['href'] and '.tif' in l['href']]
https_urls

['https://data.lpdaac.earthdatacloud.nasa.gov/lp-prod-protected/ECO_L2T_LSTE.002/ECOv002_L2T_LSTE_21241_015_10SGD_20220405T034009_0700_01/ECOv002_L2T_LSTE_21241_015_10SGD_20220405T034009_0700_01_water.tif',
 'https://data.lpdaac.earthdatacloud.nasa.gov/lp-prod-protected/ECO_L2T_LSTE.002/ECOv002_L2T_LSTE_21241_015_10SGD_20220405T034009_0700_01/ECOv002_L2T_LSTE_21241_015_10SGD_20220405T034009_0700_01_cloud.tif',
 'https://data.lpdaac.earthdatacloud.nasa.gov/lp-prod-protected/ECO_L2T_LSTE.002/ECOv002_L2T_LSTE_21241_015_10SGD_20220405T034009_0700_01/ECOv002_L2T_LSTE_21241_015_10SGD_20220405T034009_0700_01_height.tif',
 'https://data.lpdaac.earthdatacloud.nasa.gov/lp-prod-protected/ECO_L2T_LSTE.002/ECOv002_L2T_LSTE_21241_015_10SGD_20220405T034009_0700_01/ECOv002_L2T_LSTE_21241_015_10SGD_20220405T034009_0700_01_QC.tif',
 'https://data.lpdaac.earthdatacloud.nasa.gov/lp-prod-protected/ECO_L2T_LSTE.002/ECOv002_L2T_LSTE_21241_015_10SGD_20220405T034009_0700_01/ECOv002_L2T_LSTE_21241_015_10SGD_202

In [29]:
s3_urls = [l['href'] for l in granules[0]['links'] if 's3' in l['href'] and '.tif' in l['href']]
s3_urls

['s3://lp-prod-protected/ECO_L2T_LSTE.002/ECOv002_L2T_LSTE_21241_015_10SGD_20220405T034009_0700_01/ECOv002_L2T_LSTE_21241_015_10SGD_20220405T034009_0700_01_water.tif',
 's3://lp-prod-protected/ECO_L2T_LSTE.002/ECOv002_L2T_LSTE_21241_015_10SGD_20220405T034009_0700_01/ECOv002_L2T_LSTE_21241_015_10SGD_20220405T034009_0700_01_cloud.tif',
 's3://lp-prod-protected/ECO_L2T_LSTE.002/ECOv002_L2T_LSTE_21241_015_10SGD_20220405T034009_0700_01/ECOv002_L2T_LSTE_21241_015_10SGD_20220405T034009_0700_01_height.tif',
 's3://lp-prod-protected/ECO_L2T_LSTE.002/ECOv002_L2T_LSTE_21241_015_10SGD_20220405T034009_0700_01/ECOv002_L2T_LSTE_21241_015_10SGD_20220405T034009_0700_01_QC.tif',
 's3://lp-prod-protected/ECO_L2T_LSTE.002/ECOv002_L2T_LSTE_21241_015_10SGD_20220405T034009_0700_01/ECOv002_L2T_LSTE_21241_015_10SGD_20220405T034009_0700_01_LST.tif',
 's3://lp-prod-protected/ECO_L2T_LSTE.002/ECOv002_L2T_LSTE_21241_015_10SGD_20220405T034009_0700_01/ECOv002_L2T_LSTE_21241_015_10SGD_20220405T034009_0700_01_LST_err.